## Inputs
---

In [2]:
# reading data and working with arrays
import h5py, nrrd
import numpy as np

# data paths
data_path = '/groups/scicompsoft/home/fleishmang/rnr-exm-work/data/zebrafish_pair2.h5'
hdf5_file = h5py.File(data_path, 'r')
fix = hdf5_file['/fixed'][...]
mov = hdf5_file['/move'][...]

# assumed spacings (there are no provided spacings, this is a problem)
fix_spacing = np.array([1., 1., 1.])
mov_spacing = np.array([1., 1., 1.])

# # write some channels
# nrrd.write('./fix.nrrd', fix.transpose(2,1,0), compression_level=2)
# nrrd.write('./mov.nrrd', mov.transpose(2,1,0), compression_level=2)

## Alignment
---

### affine

In [3]:
%%time

# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# define alignment steps
common_kwargs = {
    'alignment_spacing':6.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(8.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':1200,
    },
}

affine_kwargs = {
    'initial_condition':'CENTER',
}

affine2_kwargs = {
    'alignment_spacing':3.0,
    'shrink_factors':(1,),
    'smooth_sigmas':(2.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':200,
    },
}

steps = [('affine', {**common_kwargs, **affine_kwargs},),]
         # ('affine', {**common_kwargs, **affine2_kwargs},),]

# align
affine = alignment_pipeline(
    fix, mov,
    fix_spacing, mov_spacing,
    steps,
)

# apply affine only
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing, mov_spacing,
    transform_list=[affine,],
)

# write results
np.savetxt('affine.mat', affine)
nrrd.write('./affine.nrrd', affine_aligned.transpose(2,1,0), compression_level=2)

# load precomputed results
affine = np.loadtxt('./affine.mat')

LEVEL:  0  ITERATION:  0  METRIC:  -0.7108752487861659
LEVEL:  0  ITERATION:  1  METRIC:  -0.7115547221543363
LEVEL:  0  ITERATION:  2  METRIC:  -0.7122312535178424
LEVEL:  0  ITERATION:  3  METRIC:  -0.7129108480073617
LEVEL:  0  ITERATION:  4  METRIC:  -0.7136718603341299
LEVEL:  0  ITERATION:  5  METRIC:  -0.7145005846970788
LEVEL:  0  ITERATION:  6  METRIC:  -0.7153592975752253
LEVEL:  0  ITERATION:  7  METRIC:  -0.716221653041901
LEVEL:  0  ITERATION:  8  METRIC:  -0.7170834545531913
LEVEL:  0  ITERATION:  9  METRIC:  -0.7179480175017818
LEVEL:  0  ITERATION:  10  METRIC:  -0.7188211152188156
LEVEL:  0  ITERATION:  11  METRIC:  -0.7197029386081181
LEVEL:  0  ITERATION:  12  METRIC:  -0.7205898293930586
LEVEL:  0  ITERATION:  13  METRIC:  -0.721476912690104
LEVEL:  0  ITERATION:  14  METRIC:  -0.7223622408655085
LEVEL:  0  ITERATION:  15  METRIC:  -0.723258899119601
LEVEL:  0  ITERATION:  16  METRIC:  -0.7241532555256587
LEVEL:  0  ITERATION:  17  METRIC:  -0.7250240345943944
LEVEL